In [173]:
# prompt: 구글드라이브 마운트하는 코드

from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from IPython.display import set_matplotlib_formats

# 레티나 디스플레이 설정
set_matplotlib_formats('retina')

cust_df = pd.read_csv('/content/drive/MyDrive/고객세분화분석/PROCESSED/cluster_df.csv')
# 병합 데이터 불러오기

import matplotlib.pyplot as plt
plt.rcParams['axes.unicode_minus'] = False

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [174]:
mg_df = pd.read_csv('/content/drive/MyDrive/고객세분화분석/PROCESSED/merge_df.csv')
mg_df = pd.merge(mg_df, cust_df[['고객ID', '클러스터']], on='고객ID', how='left')
mg_df = pd.merge(mg_df, cust_df[['고객ID', '일회성구매고객']], on='고객ID', how='left')


#클러스터 정보 기입

In [175]:
mg_df['거래날짜']= pd.to_datetime(mg_df['거래날짜'])
sorted_df = mg_df.sort_values(['고객ID','거래날짜'])
sequence_df = sorted_df.groupby('고객ID')['제품카테고리'].apply(list).tolist()

In [138]:
pip install prefixspan

In [181]:
import prefixspan

# 제공된 함수는 그대로 사용합니다.
def cluster_separate(clust_num):
    clust_target = mg_df[mg_df['일회성구매고객'] == clust_num]
    sorted_df = clust_target.sort_values(['고객ID', '거래날짜'])
    sequence_list = sorted_df.groupby('고객ID')['제품카테고리'].apply(list).tolist()
    return sequence_list

# 결과를 저장할 딕셔너리를 생성합니다.
frequent_patterns_by_cluster = {}

# 클러스터 개수만큼 for 반복문 실행
for cluster_id in mg_df['클러스터'].unique():
    # 1. 클러스터별 시퀀스 데이터 생성
    sequences = cluster_separate(cluster_id)

    # 2. PrefixSpan 실행
    ps = prefixspan.PrefixSpan(sequences)

    top_patterns = ps.topk(100)

    # 3. 결과를 데이터프레임으로 변환하여 딕셔너리에 저장
    df = pd.DataFrame(top_patterns, columns=['Support', 'Pattern'])
    frequent_patterns_by_cluster[cluster_id] = df


# --- 이후 코드는 기존과 동일합니다 ---
# 1. 딕셔너리에 저장된 데이터프레임들을 하나로 합치기
all_clusters_df = pd.concat(frequent_patterns_by_cluster, names=['Cluster', 'Rank'])

# 2. 계층적 인덱스를 일반 열(column)으로 변환
all_clusters_df = all_clusters_df.reset_index()

# 3. 분석에 불필요한 Rank 열 제거 및 유용한 정보 추가/정리
all_clusters_df = all_clusters_df.drop(columns='Rank')
all_clusters_df['Length'] = all_clusters_df['Pattern'].apply(len)

# 4. 최종적으로 보기 좋게 열 순서 정리
final_df = all_clusters_df[['Cluster', 'Support', 'Length', 'Pattern']]



In [186]:
def create_sankey_linear(df, cluster_id):
    """
    단계(Stage)를 구분하여 선형적인(Linear) 생키 다이어그램을 생성하는 함수
    cluster_id 는 클러스터 번호가 아니라 일회성 구매 고객의 여부를 의미함. <- 이후 추가 분석에서 변경되었음.
    """
    cluster_df = df[df['Cluster'] == cluster_id].copy()

    # 모든 연결(link)과 노드(node) 정보를 담을 딕셔너리
    link_data = {}

    for _, row in cluster_df.iterrows():
        pattern = row['Pattern']
        support = row['Support']
        for i in range(len(pattern) - 1):
            # 단계 정보를 포함하여 고유한 Source와 Target 이름 생성
            source_name = f"{i+1}_{pattern[i]}"
            target_name = f"{i+2}_{pattern[i+1]}"

            # (Source, Target)을 키로 사용하여 Support 값을 누적
            # 동일한 경로가 여러 패턴에서 발견될 경우 합산하기 위함
            link_key = (source_name, target_name)
            link_data[link_key] = link_data.get(link_key, 0) + support

    # 고유한 모든 노드 리스트 생성
    all_nodes = sorted(list(set([k[0] for k in link_data.keys()] + [k[1] for k in link_data.keys()])))

    # 실제 다이어그램에 표시될 노드 라벨 (단계 번호 제거)
    node_labels = [name.split('_', 1)[1] for name in all_nodes]

    # Source, Target, Value 리스트 생성
    sources = [all_nodes.index(k[0]) for k in link_data.keys()]
    targets = [all_nodes.index(k[1]) for k in link_data.keys()]
    values = [v for v in link_data.values()]

    # Sankey 다이어그램 생성
    fig = go.Figure(data=[go.Sankey(
        node=dict(
            pad=25,
            thickness=20,
            line=dict(color="black", width=0.5),
            label=node_labels,  # 화면에는 단계 번호가 없는 깔끔한 라벨 표시
            color="blue"
        ),
        link=dict(
            source=sources,
            target=targets,
            value=values
        ))])

    fig.update_layout(title_text=f"Product Category Flow top150,{cluster_id}", font_size=12)
    fig.show()

In [187]:
# 클러스터 0에 대해 수정된 함수로 다이어그램 생성
create_sankey_linear(final_df, 0)

In [188]:
create_sankey_linear(final_df, 1)

In [168]:
sorted_df = mg_df.sort_values(['고객ID', '거래날짜'])
sequence_list = sorted_df.groupby('고객ID')['제품카테고리'].apply(list).tolist()
ps = prefixspan.PrefixSpan(sequence_list)
top_patterns = ps.topk(150)
frequent_patterns = pd.DataFrame(top_patterns, columns=['Support', 'Pattern'])


# --- 이후 코드는 기존과 동일합니다 ---
# 1. 딕셔너리에 저장된 데이터프레임들을 하나로 합치기
all_df = frequent_patterns.copy()

# 2. 계층적 인덱스를 일반 열(column)으로 변환
all_df = all_df.reset_index()

# 3. 분석에 불필요한 Rank 열 제거 및 유용한 정보 추가/정리
all_df['Length'] = all_df['Pattern'].apply(len)

# 4. 최종적으로 보기 좋게 열 순서 정리
final_df = all_df[['Support', 'Length', 'Pattern']]



In [169]:
final_df

,Support,Length,Pattern
0,1323,1,[Apparel]
1,1260,1,[Nest-USA]
2,1189,2,"[Apparel, Apparel]"
3,1133,2,"[Nest-USA, Nest-USA]"
4,1103,1,[Office]
...,...,...,...
145,663,3,"[Office, Apparel, Office]"
146,661,4,"[Apparel, Apparel, Office, Office]"
147,660,6,"[Apparel, Apparel, Nest-USA, Nest-USA, Nest-US..."
148,660,4,"[Apparel, Nest-USA, Nest-USA, Office]"


In [170]:
cluster_df = final_df.copy()

# 모든 연결(link)과 노드(node) 정보를 담을 딕셔너리
link_data = {}

for _, row in cluster_df.iterrows():
    pattern = row['Pattern']
    support = row['Support']
    for i in range(len(pattern) - 1):
        # 단계 정보를 포함하여 고유한 Source와 Target 이름 생성
        source_name = f"{i+1}_{pattern[i]}"
        target_name = f"{i+2}_{pattern[i+1]}"

        # (Source, Target)을 키로 사용하여 Support 값을 누적
        # 동일한 경로가 여러 패턴에서 발견될 경우 합산하기 위함
        link_key = (source_name, target_name)
        link_data[link_key] = link_data.get(link_key, 0) + support

# 고유한 모든 노드 리스트 생성
all_nodes = sorted(list(set([k[0] for k in link_data.keys()] + [k[1] for k in link_data.keys()])))

# 실제 다이어그램에 표시될 노드 라벨 (단계 번호 제거)
node_labels = [name.split('_', 1)[1] for name in all_nodes]

# Source, Target, Value 리스트 생성
sources = [all_nodes.index(k[0]) for k in link_data.keys()]
targets = [all_nodes.index(k[1]) for k in link_data.keys()]
values = [v for v in link_data.values()]

# Sankey 다이어그램 생성
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=25,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=node_labels,  # 화면에는 단계 번호가 없는 깔끔한 라벨 표시
        color="blue"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values
    ))])

fig.update_layout(title_text=f"Product Category Flow (Linear)", font_size=12)
fig.show()